In [2]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install braindecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.4/184.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 kB 27.8 MB/s eta 0:00:00


In [ ]:
import zipfile

# Specify the path to the zip archive
zip_path = '/content/drive/MyDrive/motor3dcompressed.zip'

# Specify the directory where you want to extract the files
extract_dir = '/content/'

# Open the zip archive
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all the contents of the zip archive to the specified directory
    zip_ref.extractall(extract_dir)

print("Folder decompressed successfully.")

In [ ]:
import glob
from google.colab import files
import scipy.io
import numpy as np
# Get a list of .mat files in the current directory
file_list = glob.glob('/content/drive/MyDrive/2dMotorNormalized/*.mat')

# not merged
counter = 1

file = open('filenamechange.txt', 'w')


# Close the file

for file_path in file_list:
    file.write(file_path+" to " + str(counter)+"\n")
    mat_data = scipy.io.loadmat(file_path)
    permutation = np.random.permutation(len(mat_data['data']))

    data = mat_data['data'][permutation]
    label = mat_data['label'][0][permutation]-1
    print(label)
    training = {'data': data[:int(len(data)*0.7)], 'label': label[:int(len(data)*0.7)]}
    testing = {'data': data[int(len(data)*0.7):], 'label': label[int(len(data)*0.7):]}
    scipy.io.savemat('A0'+str(counter)+'T_P.mat', training)
    scipy.io.savemat('A0'+str(counter)+'E_P.mat', testing)
    counter += 1
file.close()

[0 0 3 2 3 3 2 3 3 1 0 1 0 1 0 0 0 1 1 0 3 3 0 1 0 1 3 2 0 1 1 1 2 2 0 2 3
 3 1 3 1 0 3 0 3 3 2 3 0 0 2 3 0 3 0 2 2 0 2 1 3 3 1 0 0 1 2 1 1 2 0 1 2 1
 3 0 0 2 0 0 2 0 0 3 1 2 1 3 2 3 2 1 2 0 0 1 0 1 0 1 1 2 1 2 2 3 1 3 2 0 0
 3 3 0 1 2 3 1 1 1 3 2 0 0 3 0 1 3 2 0 2 1 3 0 0 2 1 0 1 0 2 2 3 0 0 1 1 1
 2 1 2 1 3 2 1 2 2 1 2 3 3 3 3 0 1]
[2 1 1 1 1 3 2 0 1 3 2 3 3 3 3 1 1 0 2 0 2 2 0 1 0 2 2 1 3 0 1 1 3 1 0 0 3
 1 3 1 0 0 1 3 1 2 2 0 3 1 3 0 3 1 1 1 2 1 2 1 3 0 2 3 2 3 2 2 3 0 0 2 0 3
 1 2 2 3 3 1 1 2 2 1 2 0 2 3 3 1 3 3 3 2 1 3 0 0 0 2 1 0 0 0 0 0 3 2 2 2 0
 0 1 0 0 1 0 2 3 2 1 0 0 1 2 2 1 1 2 1 1 0 2 3 1 0 1 1 1 3 1 3 2 1 3 0 0 0
 0 0 2 2 2 0 1 3 0 0 1 3 0 1 3 3 0]
[1 3 3 3 2 0 3 0 2 2 1 0 2 0 3 0 0 1 3 0 1 1 0 2 2 0 2 3 2 1 1 2 2 1 0 0 0
 0 1 1 1 1 3 1 0 3 1 2 1 1 1 1 2 0 0 1 0 3 1 1 1 3 1 3 0 2 2 3 0 3 3 0 1 2
 0 1 2 3 2 3 3 3 0 0 0 2 1 3 2 2 3 0 0 1 2 3 1 2 1 1 0 1 3 0 1 1 1 2 3 3 2
 2 2 1 2 0 2 3 0 1 2 2 1 2 0 2 2 0 1 0 0 3 3 2 1 3 0 1 3 0 3 3 3 1 1 2 1 3
 0 3 1 3 2 0 0 0 1 1 3 2 0 0

In [ ]:
file_list = glob.glob('A*.mat')

for file_path in file_list:
    labels = scipy.io.loadmat(file_path)
    for i in labels:
        if i == -1:
            print(file_path)

In [ ]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
from braindecode.datautil import create_from_X_y
import scipy.io
import numpy as np
from skorch.helper import predefined_split
from sklearn.model_selection import train_test_split
from braindecode.models import Deep4Net
from braindecode.datautil import create_from_X_y
from braindecode import EEGClassifier
import torch
import os
import scipy.io
from skorch.callbacks import LRScheduler
from braindecode import EEGClassifier
data_dir = '/content/'


model = Deep4Net(
    in_chans=64,  # Number of input channels
    n_classes=4,  # Number of classes
    input_window_samples=640,
    final_conv_length='auto'
)

lr = 0.0625 * 0.01
weight_decay = 0
batch_size = 32
n_epochs = 100
device = 'cuda'

for i in range(1,52):
    file_name1 = None
    file_name2 = None
    if i <= 9:
        file_name1 = f'trans_A{i:02d}T_P.mat'  # Construct the file name
        file_name2 = f'trans_A{i:02d}E_P.mat'  # Construct the file name
    else:
        file_name1 = f'trans_A{i:03d}T_P.mat'
        file_name2 = f'trans_A{i:03d}E_P.mat'

    file_path1 = os.path.join(data_dir, file_name1)  # Construct the file path
    file_path2 = os.path.join(data_dir, file_name2)  # Construct the file path
    training = scipy.io.loadmat(file_path1)
    testing = scipy.io.loadmat(file_path2)

    # Assuming you have your training and testing datasets
    train_X, train_y = training['data'], (np.squeeze(training['label'])).astype(int)-1
    valid_X, valid_y = testing['data'], (np.squeeze(testing['label'])).astype(int)-1

    # Create the train-validation split using train_test_split
    train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42)

    # Create the dataset objects
    train_dataset = create_from_X_y(train_X, train_y, sfreq=250, drop_last_window=True)
    valid_dataset = create_from_X_y(valid_X, valid_y, sfreq=250, drop_last_window=True)

    # Create the predefined split using the validation dataset
    valid_split = predefined_split(valid_dataset)


    clf = EEGClassifier(
        model,
        criterion=torch.nn.NLLLoss,
        optimizer=torch.optim.AdamW,
        train_split=predefined_split(valid_dataset),
        optimizer__lr=lr,
        optimizer__weight_decay=weight_decay,
        batch_size=batch_size,
        callbacks=[
            "accuracy",
            ("lr_scheduler", LRScheduler("CosineAnnealingLR", T_max=n_epochs - 1)),
        ],
        device=None,
    )

    clf.fit(train_dataset, y=None, epochs=n_epochs)

    with open('subject'+str(i)+'_history.txt', 'w') as file:
        for j in clf.history_:
            file.write('epoch:' + str(j['epoch']) + ' train_loss:' + str(j['train_loss']) + ' train_acc:' + str(j['train_accuracy']) + ' valid_loss:' + str(j['valid_loss']) + ' valid_acc:' + str(j['valid_accuracy']) + '\n')


Streaming output truncated to the last 5000 lines.
Using data from preloaded Raw for 1 events and 640 original time points ...
Using data from preloaded Raw for 1 events and 640 original time points ...
Using data from preloaded Raw for 1 events and 640 original time points ...
Using data from preloaded Raw for 1 events and 640 original time points ...
Using data from preloaded Raw for 1 events and 640 original time points ...
Using data from preloaded Raw for 1 events and 640 original time points ...
Using data from preloaded Raw for 1 events and 640 original time points ...
Using data from preloaded Raw for 1 events and 640 original time points ...
Using data from preloaded Raw for 1 events and 640 original time points ...
Using data from preloaded Raw for 1 events and 640 original time points ...
Using data from preloaded Raw for 1 events and 640 original time points ...
Using data from preloaded Raw for 1 events and 640 original time points ...
Using data from preloaded Raw for 1 e

FileNotFoundError: ignored

In [ ]:
import shutil

# Specify the source directory where the text files are located
source_directory = '/content/'

# Specify the destination directory in your Google Drive
destination_directory = '/content/drive/MyDrive/ResultNotMergedCompressedMotor3d-CNN'

# Move all text files from the source directory to the destination directory
for file_name in os.listdir(source_directory):
    if file_name.endswith('.txt'):
        file_path = os.path.join(source_directory, file_name)
        shutil.move(file_path, destination_directory)

print("Text files moved to Google Drive successfully.")

Text files moved to Google Drive successfully.
